<a href="https://colab.research.google.com/github/NLPetroni/assignment_two/blob/main/solution_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and downloads



In [20]:
import numpy as np
import pandas as pd
import sys
import os

%cd /content
!rm -rf assignment_two &> /dev/null
!git clone https://github.com/NLPetroni/assignment_two &> /dev/null
%cd assignment_two
sys.path.append(os.getcwd())



/content
/content/assignment_two


In [114]:
from src import utils
import re
from functools import reduce
import nltk
from nltk.corpus import stopwords
from typing import List, Callable, Dict
import random
import torch

In [96]:
utils.download_data('dataset')
train_set = pd.read_csv("dataset/train_pairs.csv")
val_set = pd.read_csv("dataset/val_pairs.csv")
test_set = pd.read_csv("dataset/test_pairs.csv")

In [74]:
print(train_set.columns)
print("Total rows of the train set: {:d}".format(len(train_set)))
print("Total rows of the validation set: {:d}".format(len(val_set)))
print("Total rows of the test set: {:d}".format(len(test_set)))

Index(['Unnamed: 0', 'Claim', 'Evidence', 'ID', 'Label'], dtype='object')
Total rows of the train set: 121740
Total rows of the validation set: 7165
Total rows of the test set: 7189


In [12]:
train_set['Label'].value_counts()

SUPPORTS    89389
REFUTES     32351
Name: Label, dtype: int64

# Dataset pre-processing and conversion

In [97]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\t-]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
BAD_SYMBOLS_RE = re.compile('(-LRB-)|(-RRB-)|(-LSB-)|(-RSB-)')
INSIDE_SQAURE_BRACKETS_RE = re.compile('(-LSB-).*?(-RSB-)')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def remove_inside_square_brackets(text: str) -> str:
    return INSIDE_SQAURE_BRACKETS_RE.sub('', text)

def remove_bad_symbols(text: str) -> str:
    return BAD_SYMBOLS_RE.sub('', text)

def remove_final_tags(text: str) -> str:
   return re.sub('\.\t.*?$', '', text) 

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace('</br>', '')

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])


def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def split_text(text: str) -> List:
  return text.split()

PREPROCESSING_PIPELINE = [
                          remove_inside_square_brackets,
                          remove_bad_symbols,
                          lower,
                          remove_final_tags,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text,
                          split_text
                          ]

# Anchor method

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """

    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)


# In the evidences there is an id at the beginning of the sequence which is
# removed with the splice [:1]
train_set['Evidence'] = train_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
train_set['Claim'] = train_set['Claim'].apply(lambda txt: text_prepare(txt))

val_set['Evidence'] = val_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
val_set['Claim'] = val_set['Claim'].apply(lambda txt: text_prepare(txt))

test_set['Evidence'] = test_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
test_set['Claim'] = test_set['Claim'].apply(lambda txt: text_prepare(txt))

In [105]:
print("Before: "+str(train_copy.iloc[0]['Evidence']))
print("After: "+str(train_set.iloc[0]['Evidence']))

Before: 2	Hemsworth has also appeared in the science fiction action film Star Trek -LRB- 2009 -RRB- , the thriller adventure A Perfect Getaway -LRB- 2009 -RRB- , the horror comedy The Cabin in the Woods -LRB- 2012 -RRB- , the dark-fantasy action film Snow White and the Huntsman -LRB- 2012 -RRB- , the war film Red Dawn -LRB- 2012 -RRB- , and the biographical sports drama film Rush -LRB- 2013 -RRB- .	Star Trek	Star Trek (film)	A Perfect Getaway	A Perfect Getaway	The Cabin in the Woods	The Cabin in the Woods	Snow White and the Huntsman	Snow White and the Huntsman	Red Dawn	Red Dawn (2012 film)	Rush	Rush (2013 film)
After: ['hemsworth', 'also', 'appeared', 'science', 'fiction', 'action', 'film', 'star', 'trek', '2009', 'thriller', 'adventure', 'perfect', 'getaway', '2009', 'horror', 'comedy', 'cabin', 'woods', '2012', 'dark', 'fantasy', 'action', 'film', 'snow', 'white', 'huntsman', '2012', 'war', 'film', 'red', 'dawn', '2012', 'biographical', 'sports', 'drama', 'film', 'rush', '2013']


In [120]:
voc_evidence = [item for sublist in train_set[:]['Evidence'] for item in sublist]
voc_claim = [item for sublist in train_set[:]['Claim'] for item in sublist]
vocabulary = list(set(voc_evidence + voc_claim))

def tokenize(input: List) -> torch.Tensor:
  result = list(map(lambda x: vocabulary.index(x), input))
  return torch.tensor(result)

def detokenize(input: torch.Tensor) -> List:
  result = input.tolist()
  result = list(map(lambda x: vocabulary[x], result))
  return result

# Model definition